In [14]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import stan

try:
    import arviz as az
except ImportError as err:
    print("Please, install arviz for easy visualization of Stan models.")

import nest_asyncio
nest_asyncio.apply()

import pandas as pd

In [15]:
def simulate_samples(N : int = 100, alpha : float = 2.3, sigma : float = 2., slope : float = 4.) -> np.array:

    x = np.random.normal(size = N)
    y = alpha + slope * x + sigma * np.random.normal(size = N)
    return y

In [16]:
program_code = """

data {
    int<lower=1> N;
    vector[N] x;
    vector[N] y;
}

parameters {
    real sigma2;
    real alpha;
    real beta;
}

model {
    sigma2 ~ inv_gamma(1, 1);
    alpha ~ normal(0, 10);
    beta ~ normal(0, 10);

    for (n in 1:N) {
        y[n] ~ normal(alpha + beta * x[n], sigma2);
    }
}

"""

In [17]:
number_of_samples = 20
simulated_samples = simulate_samples(number_of_samples
)
output_data = np.zeros(shape = number_of_samples)

In [18]:
data_dict = {
    'N' : number_of_samples,
    'x' : simulated_samples,
    'y' : output_data
}

In [19]:
model = stan.build(program_code, data = data_dict)

Building...



Building: found in cache, done.Messages from stanc:
Warning in '/tmp/httpstan_wv3jl252/model_hvl6quy2.stan', line 21, column 43: A
    normal distribution is given parameter sigma2 as a scale parameter
    (argument 2), but sigma2 was not constrained to be strictly positive.
Warning in '/tmp/httpstan_wv3jl252/model_hvl6quy2.stan', line 16, column 4: Parameter
    sigma2 is given a inv_gamma distribution, which has strictly positive
    support, but sigma2 was not constrained to be strictly positive.


In [20]:
fit = model.sample(num_chains=4, num_samples = 1000, num_warmup = 500)

Sampling:   0%
Sampling:  25% (1500/6000)
Sampling:  50% (3000/6000)
Sampling:  75% (4500/6000)
Sampling: 100% (6000/6000)
Sampling: 100% (6000/6000), done.
Messages received during sampling:
  Gradient evaluation took 5.4e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.54 seconds.
  Adjust your expectations accordingly!
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: normal_lpdf: Scale parameter is -0.0415442, but must be positive! (in '/tmp/httpstan_q51bew3a/model_hvl6quy2.stan', line 21, column 8 to column 51)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception

In [21]:
results_df = fit.to_frame()

results_df.head()

parameters,lp__,accept_stat__,stepsize__,treedepth__,n_leapfrog__,divergent__,energy__,sigma2,alpha,beta
draws,,,,,,,,,,
0,44.077321,0.973673,0.433191,3.0,7.0,0.0,-42.276986,0.066399,0.009133,0.002238
1,44.520051,0.943504,0.410897,2.0,3.0,0.0,-44.274344,0.058469,-0.007783,0.003784
2,44.662695,0.997777,0.394304,3.0,7.0,0.0,-40.769182,0.065283,0.002154,0.000875
3,42.614621,0.877817,0.434894,2.0,3.0,0.0,-42.301011,0.073368,0.003911,-0.005824
4,44.925064,1.000000,0.433191,2.0,3.0,0.0,-44.044013,0.058236,0.006657,-0.002713
